## 6 April 8pm-Changed ML to include 'generic' time of day rendering in numerical fashion(HHMM integer format)  - also saving individual pikle files  

Import Libraries

In [1]:
from sqlalchemy import create_engine    
import pandas as pd
from datetime import date, datetime
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
import pickle
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Creating a connection to the database

In [2]:
APIKEY=""
             
# create SQL engine                         
engine = create_engine('mysql+mysqlconnector://admin:@dbike..us-east-1.rds.amazonaws.com:3306/dbikes') 

Running Query and loading it into dataframe to test connection .

In [3]:
try: 
    conn = engine.connect()
    results = conn.execute('SELECT  a.number AS StationNumber, (a.last_update DIV 300 * 300) AS Time, MAX(a.available_bikes) AS BikesAvailable, MAX(a.available_bike_stands) AS BikeStandsAvailable, MAX(w.temperature) AS Temp, MAX(w.humity) AS Humity, MAX(w.wind_speed) AS Wind FROM dbikes.availability AS a, dbikes.weather AS w WHERE (a.last_update DIV 300 * 300) = (w.timestamp DIV 300 * 300) GROUP BY a.number, (a.last_update DIV 300 * 300) ORDER BY a.number, (a.last_update DIV 300 * 300);')
    df = pd.DataFrame(results.fetchall())
    rows = results.fetchall()
except:
    print(traceback.format_exc())
    # return "error in get_stations", 404


df.shape

/var/folders/b2/srz1ml_95kjgbnkt6h1zv83w0000gn/T/ipykernel_15489/1865149595.py:3: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  results = conn.execute('SELECT  a.number AS StationNumber, (a.last_update DIV 300 * 300) AS Time, MAX(a.available_bikes) AS BikesAvailable, MAX(a.available_bike_stands) AS BikeStandsAvailable, MAX(w.temperature) AS Temp, MAX(w.humidity) AS Humidity, MAX(w.wind_speed) AS Wind FROM dbikes.availability AS a, dbikes.weather AS w WHERE (a.last_update DIV 300 * 300) = (w.timestamp DIV 300 * 300) GROUP BY a.number, (a.last_update DIV 300 * 300) ORDER BY a.nu

(555864, 7)

Print the number of duplicates, without the original rows that were duplicated

In [4]:
print('Number of duplicates, without first rows is: ', df.duplicated().sum())

Number of duplicates, without first rows is:  0


Adding DAY_OF_WEEK(Weekday Saturday Sunday) Category 

In [5]:

#Creating empty array to add day_of_week and time_of_day values
day_of_week = []
time_of_day = []
for x in df.index:
    week_day = datetime.weekday(date.fromtimestamp(df.loc[x, 'Time']))
    time_of_day.append(int(datetime.fromtimestamp(df.loc[x, 'Time']).strftime('%H%M')))
    if week_day == 6:
       day = week_day
    elif week_day == 5:
       day = week_day
    elif week_day == 4:
       day = week_day
    else:
       day = 0
    day_of_week.append(day)

df['Weekday'] = day_of_week
df['Timeday'] = time_of_day
df = df.drop('Time',axis=1)

df.head(148)

,StationNumber,BikesAvailable,BikeStandsAvailable,Temp,Humidity,Wind,Weekday,Timeday
0,1,0,31,279.73,89,5.14,4,450
1,1,0,31,279.93,88,4.12,4,510
2,1,1,30,280.21,87,6.69,4,645
3,1,4,27,280.33,88,6.69,4,705
4,1,6,26,280.28,88,5.66,4,715
...,...,...,...,...,...,...,...,...
143,1,20,11,279.84,69,5.66,6,0
144,1,20,11,279.84,67,5.66,6,20
145,1,20,11,279.84,65,4.63,6,40
146,1,20,11,279.69,67,4.63,6,50


Creating variables for Model's 'Targets & Training Variables

In [6]:

#Columns classification for Logistic Regression Model 
variables_training = ['Temp', 'Humidity', 'Wind', 'Weekday', 'Timeday']
variable_targets = ['BikesAvailable', 'BikeStandsAvailable']


In [7]:
# Create list of Stations and Dictionary of Models
stations = []
stations.append(df['StationNumber'].unique())
stationModels = {}

In [8]:
# Loop to do one for each Station
for station in stations[0]:

    # Filter by Target Station
    data = df.query('StationNumber == ' + str(station))

    #Setting Model's target data
    y = data[variable_targets]

    #Setting Model's variables data
    x = data[variables_training]
    
    #Splitting target data and variables data attributes into train and test tuples
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    
    # Initialize the MLPRegressor
    stationModels[station] = MLPRegressor(hidden_layer_sizes=(100, 100), activation='relu', solver='adam', random_state=42)

    #Run the Model with train tuples
    stationModels[station].fit(x_train, y_train)

    # Serialize model object into a file called model.pkl on disk using pickle
    pkl_name = 'station' + str(station) + '.pkl'
    with open(pkl_name, 'wb') as handle:
        pickle.dump(stationModels[station], handle, pickle.HIGHEST_PROTOCOL)

In [9]:
# predictions = linModel.predict(x_test[0:1])
# # predictions

# # Serialize model object into a file called model.pkl on disk using pickle
# with open('model.pkl', 'wb') as handle:
#     pickle.dump(linModel, handle, pickle.HIGHEST_PROTOCOL)


In [2]:
!pip freeze > requirements.txt